# Fallabella's Sneakers Analysis

The objective is to gather information about the characteristics of sneakers, including their prices, discounts, and original costs, in order to examine potential emerging correlations between discounts, pricing, and specific attributes. This analysis aims to validate if it is possible to:

- Predict product pricing
- Predict/determine discount proportions (for instances when people have Falabella's credit card and when they do not)
- Descriptive analysis of the products

### 1. Understanding web page structure



In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?facetSelected=true&f.derived.variant.sellerId=FALABELLA")

In [ ]:
# create a function to get the brands of the products
def getproduct_brand():
    # reload the page
    driver.refresh()
    brands = driver.find_elements(By.CLASS_NAME, "title-rebrand")
    for brand in brands:
        print(brand.text)

print(getproduct_brand())

In [ ]:
# create a function to get the names of the products
def getproduct_name():
    names = driver.find_elements(By.CLASS_NAME, "pod-subTitle")
    for name in names:
        print(name.text)
        

print(getproduct_name())


In [ ]:
# getting price 

def getproduct_price():
    prices = driver.find_elements(By.CLASS_NAME, "jsx-2112733514") 
    counter = 0
    
    for price in prices:
        print(price.text)
        # print product counter
        counter += 1
        print(f"product {counter}")
        # select li elements
        price_items = price.find_elements(By.TAG_NAME, "li")
        # iterate over the li elements
        
        for item in price_items:

            if item.get_attribute("data-cmr-price"):
                cmr_price = item.get_attribute("data-cmr-price")
                print("precio cmr:" + cmr_price)
            if item.get_attribute("data-event-price"):
                event_price = item.get_attribute("data-event-price")
                print("precio descuento:" + event_price)
            if item.get_attribute("data-normal-price"):
                full_price = item.get_attribute("data-normal-price")
                print("precio full:" + full_price)       
        if None in prices:
            continue
        
        counter += 1
    
                    

print(getproduct_price())

### 1.1 looking at sneakers details


In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

def get_product_details_url():
    # get the names of the products
    names = driver.find_elements(By.CLASS_NAME, "pod-subTitle")
    element = names[1]
    # create an action chain object to click on the element and expand the product details
    actions = ActionChains(driver)
    actions.move_to_element(element).click().perform()
    # wait to the element to be clickable
    driver.implicitly_wait(10)
    buttons = driver.find_elements(By.CLASS_NAME, "mkp-swatchButton-collapseButton")
    for button in buttons:
        if button.text == "ver más":
             button.click()
            
    
    table = driver.find_element(By.CLASS_NAME, "specification-table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        categories = row.find_element(By.TAG_NAME, "td")
        print(categories.text) # this will be columns of the data frame 
        values = row.find_elements(By.TAG_NAME, "td")
        print(values[1].text) # this will be the values of the data frame according to the columns
           
    # then, we have to go back to the previous page to get the next product details
    
get_product_details_url()

## 2. Implementation


### 2.1 Collecting all product details and page links
- detail_links: Save links of each product.
- page_links: Save page links that contain all products.

In [8]:
# get href of the atribute a in the actual page
def get_href():
    detail_links = []
    hrefs = driver.find_elements(By.CLASS_NAME, "pod-2_GRID")
    for href in hrefs:
        detail_links.append(href.get_attribute("href"))
    return detail_links

detail_links = get_href()


### 3.  Data Frame Test 1.

In [154]:
driver = webdriver.Chrome()

In [142]:
temp_df = pd.DataFrame(columns=[
        'Brand', 'Name', 'CMR Price', 'Event Price', 'Full Price', 'Gender',
        'Type', 'Fit', 'Material', 'Material forro', 'Sole Material',
        'Interior Material', 'Season', 'Made In', 'Discipline', 'Insole Material'
    ])

In [ ]:
def create_df():
    # create a list to store the data
    
    columns = [
        'Brand', 'Name', 'CMR Price', 'Event Price', 'Full Price', 'Gender',
        'Type', 'Fit', 'Material', 'Material forro', 'Sole Material',
        'Interior Material', 'Season', 'Made In', 'Discipline', 'Insole Material'
    ]
    
    
    for link in pages_links:
        print(link)
        link_per = link
        driver.get(link)
        sections = driver.find_elements(By.CLASS_NAME, "pod-link")
        print(len(sections))
        
        for section in sections:
            data = {
                'Brand': None, 'Name': None, 'CMR Price': None, 'Event Price': None,
                'Full Price': None, 'Gender': None, 'Type': None, 'Fit': None, 
                'Material': None, 'Material forro': None, 'Sole Material': None,
                'Interior Material': None, 'Season': None, 'Made In': None, 
                'Discipline': None, 'Insole Material': None
            }
            

            genero = tipo = horma = material = material_suela = material_interior = temporada = hecho_en = disciplina = material_plantilla = brand = name =  None
            # get the brand of the product
            brand = section.find_element(By.CLASS_NAME, "title-rebrand").text
            print("brand: " + brand)
            # get the name of the product
            name = section.find_element(By.CLASS_NAME, "pod-subTitle").text
            print("name: " + name)
            # get the price of the product
            

            cmr_price = None
            event_price = None
            full_price = None

            # Intenta encontrar los tres precios, si es posible
            try:
                cmr_price = section.find_element(By.CLASS_NAME, "prices-0").get_attribute("data-cmr-price")
                print("cmr price: " + str(cmr_price))
            except Exception:
                pass  # Si no se encuentra, simplemente continúa

            try:
                event_price = section.find_element(By.CLASS_NAME, "prices-1").get_attribute("data-event-price")
                print("event price: " + str(event_price))
            except Exception:
                pass  # Si no se encuentra, simplemente continúa

            try:
                full_price = section.find_element(By.CLASS_NAME, "prices-2").get_attribute("data-normal-price")
                print("full price: " + str(full_price))
            except Exception:
                pass  # Si no se encuentra, simplemente continúa

            # Si no se encontraron los tres precios, intenta encontrar la configuración de un solo precio o dos precios
            if not cmr_price and not event_price and not full_price:
                try:
                    full_price = section.find_element(By.CLASS_NAME, "prices-0").get_attribute("data-internet-price")
                    print("full price: " + str(full_price))
                except Exception:
                    pass  # Si no se encuentra, simplemente continúa

                try:
                    event_price = section.find_element(By.CLASS_NAME, "prices-0").get_attribute("data-event-price")
                    print("event price: " + str(event_price))
                except Exception:
                    pass  # Si no se encuentra, simplemente continúa

                try:
                    full_price = section.find_element(By.CLASS_NAME, "prices-1").get_attribute("data-normal-price")
                    print("full price: " + str(full_price))
                except Exception:
                    pass  # Si no se encuentra, simplemente continúa

                
                
            
            
            # get the href of the product
            href = section.get_attribute("href")
            driver.get(href)
            # print the href
            print(href)
            buttons = driver.find_elements(By.CLASS_NAME, "mkp-swatchButton-collapseButton")
            for button in buttons:
                if button.text == "ver más":
                    button.click()
                    print("button clicked")
            
            
            # get the product details
            table = driver.find_element(By.CLASS_NAME, "specification-table")
            print(table)
            rows = table.find_elements(By.TAG_NAME, "tr")
            print(rows)
            for row in rows:
                
                try:
                    # Encuentra todas las celdas en la fila
                    values = row.find_elements(By.TAG_NAME, "td")
                    if len(values) < 2:
                        # Si no hay suficientes celdas, continua con la siguiente fila
                        continue

                    # Obtén el atributo de la primera celda y conviértelo a minúsculas
                    attribute_name = values[0].text.lower().strip()
                    # Verifica si la segunda celda tiene texto antes de asignarlo
                    attribute_value = values[1].text.strip() if values[1].text else None
                    print(f"{attribute_name}: {attribute_value}")

                    # Asigna el valor del atributo a la variable correspondiente
                    if attribute_name == "género":
                        genero = attribute_value
                        print("la variable genero es: " + genero)
                    elif attribute_name == "tipo":
                        tipo = attribute_value
                    elif attribute_name == "horma":
                        horma = attribute_value
                    elif attribute_name == "material":
                        material = attribute_value
                    elif attribute_name == "material del forro":
                        material_forro = attribute_value
                    elif attribute_name == "material de la suela":
                        material_suela = attribute_value
                    elif attribute_name == "material del interior":
                        material_interior = attribute_value
                    elif attribute_name == "temporada":
                        temporada = attribute_value
                    elif attribute_name == "hecho en":
                        hecho_en = attribute_value
                    elif attribute_name == "disciplina":
                        disciplina = attribute_value
                        print("disciplina essss: " + disciplina)
                    elif attribute_name == "material de la plantilla":
                        material_plantilla = attribute_value
                        print("material plantilla esssss: " + material_plantilla)

                except Exception as e:
                    print(f"An exception occurred: {e}")
                
            # append the data to the list
            data.append({
                'Brand': brand,
                'Name': name,
                'CMR Price': cmr_price,
                'Event Price': event_price,
                'Full Price': full_price,
                'Gender': genero,
                'Type': tipo,
                'Fit': horma,
                'Material': material,
                'Material forro': material_forro,
                'Sole Material': material_suela,
                'Interior Material': material_interior,
                'Season': temporada,
                'Made In': hecho_en,
                'Discipline': disciplina,
                'Insole Material': material_plantilla
            })   # create a dataframe from the data
            print(" se ha creado un dataframe ")
            
            # Append the current product's data to the CSV file
            # Append current product's data to the CSV file
            pd.DataFrame([data], columns=columns).to_csv("../ws-falabella", mode='a', header=False, index=False)
            print("Data appended to CSV:", data)

            print(data)  # Imprime el diccionario de datos
            driver.back()  # Regresa a la página principal
            
                
    df = pd.DataFrame(data)
    
    return df

df = create_df()
            

In [ ]:
df = create_df(temp_df)

### 2.2    Data Frame Test 2

In [94]:
driver = webdriver.Chrome()
driver.get("https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA")

In [48]:
pages = driver.find_elements(By.CLASS_NAME, "jsx-1389196899")
print(len(pages))

32


In [88]:
pages_links = []
pages_links.append("https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA")
for i in range(2, len(pages) + 1):
    link = f"https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page={i}"
    pages_links.append(link)
print(pages_links)

['https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=2', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=3', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=4', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=5', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=6', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=7', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.

In [ ]:
def get_all_links():
    all_links = []
    for link in pages_links:
        print(link)
        driver.get(link)
        boxes = driver.find_elements(By.CLASS_NAME, "pod-link")
        print(len(boxes))
        for box in boxes:
            box.get_attribute("href")
            print(box.get_attribute("href"))
            all_links.append(box.get_attribute("href"))
        
        print(all_links)
    return all_links
            
    

all_links = get_all_links()
# make all links a csv file
import pandas as pd

df = pd.DataFrame(all_links)
df.to_csv("all_links.txt", index=False)

In [91]:
# remove all " from the links
def clean_links():
    clean_links = []
    for link in links:
        clean_link = link.replace('"', "")
        clean_links.append(clean_link)
    return clean_links

clean_links = clean_links()
print(clean_links)

# save the clean links in a txt file
def save_clean_links():
    with open('clean_links.txt', 'w') as file:
        for link in clean_links:
            file.write(link + "\n")

save_clean_links()

['https://www.falabella.com.co/falabella-co/product/72889166/Tenis-Adidas-para-Mujer-Moda-Court-Platform/72889168']


In [96]:
# read clean links from the txt file and store them in a list
def read_clean_links():
    links = []
    with open('clean_links.txt', 'r') as file:
        for link in file:
            links.append(link)
    return links

links = read_clean_links()

In [81]:
def pricing(driver):
    cmr_price = event_price = full_price = None  # Initialize the variables

    # Attempt to fetch each type of price, handle exceptions if elements are not found
    try:
        cmr_elements = driver.find_elements(By.CLASS_NAME, "prices-0")
        if cmr_elements:
            cmr_price = cmr_elements[0].get_attribute("data-cmr-price")
    except Exception:
        pass  # If not found, simply continue

    try:
        event_elements = driver.find_elements(By.CLASS_NAME, "prices-1")
        if event_elements:
            event_price = event_elements[0].get_attribute("data-event-price")
    except Exception:
        pass  # If not found, simply continue

    try:
        full_elements = driver.find_elements(By.CLASS_NAME, "prices-2")
        if full_elements:
            full_price = full_elements[0].get_attribute("data-normal-price")
    except Exception:
        pass  # If not found, simply continue

    # Fallback strategies for other price configurations
    try:
        if not full_price:
            full_elements = driver.find_elements(By.CLASS_NAME, "prices-0")
            if full_elements:
                full_price = full_elements[0].get_attribute("data-internet-price")
    except Exception:
        pass  # If not found, simply continue

    return cmr_price, event_price, full_price

In [82]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

def create_df(links):
    
    all_data = pd.DataFrame()
    
    for link in links:
        print("Scraping:", link)
        driver.get(link)
        
        try:
            name = driver.find_element(By.CLASS_NAME, "product-name").text
            rating = driver.find_element(By.CLASS_NAME, "_text_17o93_5").text
            code = driver.find_element(By.CLASS_NAME, "jsx-3410277752").text
            table = driver.find_element(By.CLASS_NAME, "specification-table")
            rows = table.find_elements(By.TAG_NAME, "tr")
            
            prices = pricing(driver)
            
            data = {
                'Name': [name],
                'Rating': [rating],
                'Code': [code],
                'CMR Price': [prices[0]],
                'Event Price': [prices[1]],
                'Full Price': [prices[2]]
            }


            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if cells and len(cells) > 1:
                    attribute_name = cells[0].text.strip().lower().replace(" ", "_")
                    attribute_value = cells[1].text.strip()
                    data[attribute_name] = [attribute_value]

            temp_df = pd.DataFrame(data)
            all_data = pd.concat([all_data, temp_df], ignore_index=True)

        except Exception as e:
            print(f"Error scraping {link}: {e}")

    driver.quit()
    return all_data


In [98]:
driver = webdriver.Chrome()
first_df = create_df(links=read_clean_links())
first_df

Scraping: https://www.falabella.com.co/falabella-co/product/72778441/Tenis-Skechers-para-Mujer-Moda-Slip-Ins-Ultra-Flex-3.0/72778443

Scraping: https://www.falabella.com.co/falabella-co/product/42790543/Tenis-Under-Armour-Mujer-Running-Surge-3/42790553

Scraping: https://www.falabella.com.co/falabella-co/product/72804662/Tenis-Reebok-para-Mujer-Cross-Training-Nanoflex-Tr-2-Tenis-para-Mujer-de-Cross-Training-Color-Azul/72804665

Scraping: https://www.falabella.com.co/falabella-co/product/72875575/Tenis-Nike-para-Hombre-Running-Revolution7/72875577

Scraping: https://www.falabella.com.co/falabella-co/product/72778358/Tenis-Skechers-para-Mujer-Moda-Go-Walk-Air-2.0-Clas/72778360

Scraping: https://www.falabella.com.co/falabella-co/product/72784000/Tenis-Nike-para-Mujer-Running-Revolution-6-Nn/72784006

Scraping: https://www.falabella.com.co/falabella-co/product/56488639/Tenis-Nike-Hombre-Cross-Training-Renew-Retaliation-4/56488644

Scraping: https://www.falabella.com.co/falabella-co/produc

,Name,Rating,Code,CMR Price,Event Price,Full Price,nombre_comercial,marca,modelo,tipo,...,temporada,material_del_interior,hecho_en,transpirabilidad_de_la_tela,altura_de_la_plataforma,tipo_de_ajuste,forma_de_la_punta,altura_del_tacón,tipo_de_suela,registro_sic
0,Tenis Skechers para Mujer Moda Slip Ins Ultra ...,(0),Código: 72778443,233.890,272.890,389.900,Ultraflex - Slip In,Skechers,149709MVE,Tenis moda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tenis Under Armour Mujer Running Surge 3,4.9 (17),Código: 42790553,216.890,247.890,309.900,Surge 3,Under Armour,3024894-602,Tenis deportivos,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tenis Reebok para Mujer Cross Training Nanofle...,4 (2),Código: 72804665,272.990,311.990,389.990,Nanoflex Tr 2,Reebok,100033772,Tenis,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tenis Nike para Hombre Running Revolution7,(0),Código: 72875577,None,None,344.990,Revolution,Nike,FB2207-001-BLK,Tenis,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tenis Skechers para Mujer Moda Go Walk Air 2.0...,(0),Código: 72778360,227.890,265.890,379.900,Go Walk,Skechers,124362NVPR,Tenis moda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,Tenis New Balance para Hombre Moda 550,5 (3),Código: 72735458,None,None,859.990,550,New Balance,BB550VTB,Tenis moda,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1519,Tenis Skechers para Mujer Moda Summits Perfect...,(0),Código: 72890296,179.990,199.990,279.900,Summits,Skechers,149523WPWP,Tenis moda,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1520,Tenis Adidas Mujer Cross training Rapidmove Ad...,4.6 (41),Código: 72712203,454.990,519.990,649.950,Rapidmove Adv Train,Adidas,HP3270,Tenis deportivos,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1521,Tenis Adidas para Hombre Running Adizero SL,4.8 (778),Código: 72889124,None,None,699.990,Adizero SL,Adidas,IG8200,Tenis deportivos,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
first_df.to_csv("first_df.csv", index=False)

In [100]:
first_df

,Name,Rating,Code,CMR Price,Event Price,Full Price,nombre_comercial,marca,modelo,tipo,...,temporada,material_del_interior,hecho_en,transpirabilidad_de_la_tela,altura_de_la_plataforma,tipo_de_ajuste,forma_de_la_punta,altura_del_tacón,tipo_de_suela,registro_sic
0,Tenis Skechers para Mujer Moda Slip Ins Ultra ...,(0),Código: 72778443,233.890,272.890,389.900,Ultraflex - Slip In,Skechers,149709MVE,Tenis moda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tenis Under Armour Mujer Running Surge 3,4.9 (17),Código: 42790553,216.890,247.890,309.900,Surge 3,Under Armour,3024894-602,Tenis deportivos,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tenis Reebok para Mujer Cross Training Nanofle...,4 (2),Código: 72804665,272.990,311.990,389.990,Nanoflex Tr 2,Reebok,100033772,Tenis,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tenis Nike para Hombre Running Revolution7,(0),Código: 72875577,None,None,344.990,Revolution,Nike,FB2207-001-BLK,Tenis,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tenis Skechers para Mujer Moda Go Walk Air 2.0...,(0),Código: 72778360,227.890,265.890,379.900,Go Walk,Skechers,124362NVPR,Tenis moda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,Tenis New Balance para Hombre Moda 550,5 (3),Código: 72735458,None,None,859.990,550,New Balance,BB550VTB,Tenis moda,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1519,Tenis Skechers para Mujer Moda Summits Perfect...,(0),Código: 72890296,179.990,199.990,279.900,Summits,Skechers,149523WPWP,Tenis moda,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1520,Tenis Adidas Mujer Cross training Rapidmove Ad...,4.6 (41),Código: 72712203,454.990,519.990,649.950,Rapidmove Adv Train,Adidas,HP3270,Tenis deportivos,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1521,Tenis Adidas para Hombre Running Adizero SL,4.8 (778),Código: 72889124,None,None,699.990,Adizero SL,Adidas,IG8200,Tenis deportivos,...,Toda temporada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
